script to use the dalitz plot method outlined by matt shepard to calculate the efficiency of the kstar rejection in the KKPi phasespace

The idea would be to correct the data by this as opposed to the phasespace acceptance, as this just handles the K* rejection and not the overall efficiency (which may not actually model the f1 well, unlike the signal MC)

In [2]:
import ROOT

In [6]:
mass_low = 1.0 # GeV
mass_high = 2.5
step_size = 0.01

kstar_mass_upper = 1.0
kstar_mass_lower = 0.8

kaon_mass = 0.493677
kshort_mass = 0.497611
pion_mass = 0.139570


In [4]:
def get_weight_per_mass(mass_bin):
    #TODO cast the list as a c array
    phasespace_centermass = ROOT.TLorentzVector(0, 0, 0, mass_bin)
    phasespace_generator = ROOT.TGenPhaseSpace()
    phasespace_generator.SetDecay(phasespace_centermass, 3, [kaon_mass, kshort_mass, pion_mass])

    total = 0
    passed = 0

    for i in range(100000):
        weight = phasespace_generator.Generate()
        kaon = phasespace_generator.GetDecay(0)
        kshort = phasespace_generator.GetDecay(1)
        pion = phasespace_generator.GetDecay(2)

        neutral_kstar_mass = (kaon + pion).M()
        charged_kstar_mass = (kshort + pion).M()

        total += weight

        if neutral_kstar_mass > kstar_mass_lower and neutral_kstar_mass < kstar_mass_upper:
            passed += weight

    return passed / total

In [7]:
test_weight = get_weight_per_mass(1.35)

TypeError: bool TGenPhaseSpace::SetDecay(TLorentzVector& P, int nt, const double* mass, const char* opt = "") =>
    TypeError: could not convert argument 3 (could not convert argument to buffer or nullptr)